# Feature Engineering | Data Preparation

This notebook includes some final data cleaning and also the calculation of the Relative Strength Index (RSI) and Lagged Variables to be added to the dataset for addtional feature engineered variables.

## Loading Data and Some quick cleaning

It was noted that for the 6-month dataset, the full values for the stock prices were included accidentally making this a dataset with many missing values (170 vs 990). As the dataset is chronologically ordered, and was deisgned so that the first part of the dataset would include the twitter sentiment, this is easily fixable with a quick index split.

In [ ]:
#loading datasets
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler
drive.mount('/content/drive')
file_path_1 = '/content/drive/MyDrive/FILE_PATH/scaled_data_1_final.csv'  #Replace FILE_PATH with the actual file destination or upload directly to notebook -THIS IS THE DATASET FROM ASSIGNMENT 1
file_path_2 = '/content/drive/MyDrive/FILE_PATHscaled_data_2_final.csv'  #Replace FILE_PATH with the actual file destination or upload directly to notebook -THIS IS THE DATASET FROM ASSIGNMENT 1

df_1 = pd.read_csv(file_path_1)
df_2 = pd.read_csv(file_path_2)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_1.head()

,Time_UTC,Close_Avg,Close_UTC_00,Close_ET_4pm,Close Value Apple,Close Value SPDR S&P 500 ETF Trust,Close Value Coinbase
0,2022-01-01,0.547344,0.533485,0.551347,0.478597,0.568411,0.889392
1,2022-01-02,0.555135,0.551399,0.551912,0.478597,0.568411,0.889392
2,2022-01-03,0.549743,0.541158,0.539575,0.519035,0.581613,0.884052
3,2022-01-04,0.539964,0.531334,0.542050,0.497996,0.580845,0.880411
4,2022-01-05,0.537233,0.517471,0.539240,0.454463,0.536822,0.816005


In [ ]:
df_2.head()

,Time_UTC,avg_sentiment_score,Close_Avg,Close_UTC_00,Close_ET_4pm,Close Value Apple,Close Value SPDR S&P 500 ETF Trust,Close Value Coinbase
0,2022-01-11,0.210110,0.460943,0.445929,0.453174,0.455920,0.543399,0.828141
1,2022-01-12,0.260047,0.478779,0.466867,0.486503,0.460018,0.549496,0.817906
2,2022-01-13,0.222628,0.490983,0.481957,0.478203,0.429599,0.518339,0.791731
3,2022-01-14,0.253282,0.473019,0.466895,0.477653,0.437614,0.519251,0.799013
4,2022-01-15,0.253282,0.480911,0.472452,0.482608,0.437614,0.519251,0.799013


In [ ]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 8 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Time_UTC                            170 non-null    object 
 1   avg_sentiment_score                 170 non-null    float64
 2   Close_Avg                           990 non-null    float64
 3   Close_UTC_00                        990 non-null    float64
 4   Close_ET_4pm                        990 non-null    float64
 5   Close Value Apple                   990 non-null    float64
 6   Close Value SPDR S&P 500 ETF Trust  990 non-null    float64
 7   Close Value Coinbase                990 non-null    float64
dtypes: float64(7), object(1)
memory usage: 62.0+ KB


In [ ]:
#snipping to get the desired dataset
df_2 = df_2.iloc[:170]

## Relative Strength Index
The RSI is often used in similar tasks due to its potential of highlighting the strengths and weaknesses of a stock and so it was included in this project. It was decided to perform this on the Close Price at UTC midnight, more information about why can be found in the report. Resources for implementing the RSI in python sourced [here](https://mayerkrebs.com/relative-strength-index-rsi-in-python/).


Implementation in Assignment 1
```
#Creating the RSI
change = close_df["Close_UTC_00"].diff().fillna(0) #filling any nans with 0
change.dropna(inplace=True) # shouldn't be any

# Saving copies as backups
change_u = change.copy()
change_d = change.copy()

#initialinzing
change_u[change_u < 0] = 0
change_d[change_d > 0] = 0
change_d = abs(change_d) #keeping absolute values for change down

# Double checking sum of ups and downs equals original days
assert change.equals(change_u - change_d) #- as we took absolute value

# Calculate the rolling average for 14 days RSI
rolling_window = 14 #adjustable
avg_up = change_u.rolling(rolling_window, min_periods=1).mean()
avg_down = change_d.rolling(rolling_window, min_periods=1).mean()

#calculating
eps = 1e-10 # to avoid dividing by zero
rsi = 100 * avg_up / (avg_up + avg_down + eps)

#Adding to dataframe
close_df['RSI'] = rsi

#checking
close_df[["Close_UTC_00", "RSI"]].head(10)
```

Slightly adapted here to make more modular.

In [ ]:
#RSI Calculator
def rsi_features(series, rolling_window=14):
  change = series.diff().fillna(0) #filling any nans with 0
  change.dropna(inplace=True) # shouldn't be any

  #saving copies as backups
  change_u = change.copy()
  change_d = change.copy()

  #initialinzing
  change_u[change_u < 0] = 0
  change_d[change_d > 0] = 0
  change_d = abs(change_d) #keeping absolute values for change down

  #Double checking sum of ups and downs equals original days
  assert change.equals(change_u - change_d) #- as we took absolute value

  #Calculate the rolling average for 14 days RSI
  avg_up = change_u.rolling(rolling_window, min_periods=1).mean()
  avg_down = change_d.rolling(rolling_window, min_periods=1).mean()

  #calculating
  eps = 1e-10 # to avoid zero division
  rsi = 100 * avg_up / (avg_up + avg_down + eps)

  return rsi

#defining lag features
def lagged_features(df, columns, lags=[1,3,7]):
  for col in columns:
    for lag in lags:
      df[f"{col}_lag{lag}"] = df[col].shift(lag)

  return df

For DF 1 we will be created the RSI and Lagged Features for Close_00_UTC

For DF 2 we will also be created Lagged Features for Twitter Sentiment.

In [ ]:
#applying RSI
df_1['RSI'] = rsi_features(df_1['Close_UTC_00'])
df_2['RSI'] = rsi_features(df_2['Close_UTC_00'])

#apply min-max scaler to RSI to be consistent with other variables
scaler = MinMaxScaler()
df_1['RSI'] = scaler.fit_transform(df_1[['RSI']])
df_2['RSI'] = scaler.fit_transform(df_2[['RSI']])

#Appling Lag
lag_columns_1 = ["Close_UTC_00", "RSI"]
lag_columns_2 = ["Close_UTC_00", "avg_sentiment_score","RSI"]
df_1 = lagged_features(df_1, lag_columns_1)
df_2 = lagged_features(df_2, lag_columns_2)


In [ ]:
df_1.head()

,Time_UTC,Close_Avg,Close_UTC_00,Close_ET_4pm,Close Value Apple,Close Value SPDR S&P 500 ETF Trust,Close Value Coinbase,RSI,Close_UTC_00_lag1,Close_UTC_00_lag3,Close_UTC_00_lag7,RSI_lag1,RSI_lag3,RSI_lag7
0,2022-01-01,0.547344,0.533485,0.551347,0.478597,0.568411,0.889392,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-02,0.555135,0.551399,0.551912,0.478597,0.568411,0.889392,1.000000,0.533485,NaN,NaN,0.000000,NaN,NaN
2,2022-01-03,0.549743,0.541158,0.539575,0.519035,0.581613,0.884052,0.636253,0.551399,NaN,NaN,1.000000,NaN,NaN
3,2022-01-04,0.539964,0.531334,0.542050,0.497996,0.580845,0.880411,0.471683,0.541158,0.533485,NaN,0.636253,0.0,NaN
4,2022-01-05,0.537233,0.517471,0.539240,0.454463,0.536822,0.816005,0.345547,0.531334,0.551399,NaN,0.471683,1.0,NaN


In [ ]:
#drop any Nan rows (caused due to rolling)
df_1.dropna(inplace=True)
df_2.dropna(inplace=True)

In [ ]:
df_1.head()

,Time_UTC,Close_Avg,Close_UTC_00,Close_ET_4pm,Close Value Apple,Close Value SPDR S&P 500 ETF Trust,Close Value Coinbase,RSI,Close_UTC_00_lag1,Close_UTC_00_lag3,Close_UTC_00_lag7,RSI_lag1,RSI_lag3,RSI_lag7
7,2022-01-08,0.457446,0.448601,0.450442,0.429417,0.525828,0.808318,0.172559,0.476972,0.517471,0.533485,0.220912,0.345547,0.000000
8,2022-01-09,0.456446,0.429663,0.451505,0.429417,0.525828,0.808318,0.150561,0.448601,0.521923,0.551399,0.172559,0.397311,1.000000
9,2022-01-10,0.457169,0.453001,0.438294,0.429599,0.523044,0.778704,0.265888,0.429663,0.476972,0.541158,0.150561,0.220912,0.636253
10,2022-01-11,0.460943,0.445929,0.453174,0.455920,0.543399,0.828141,0.255382,0.453001,0.448601,0.531334,0.265888,0.172559,0.471683
11,2022-01-12,0.478779,0.466867,0.486503,0.460018,0.549496,0.817906,0.333373,0.445929,0.429663,0.517471,0.255382,0.150561,0.345547


In [ ]:
df_2.head()

,Time_UTC,avg_sentiment_score,Close_Avg,Close_UTC_00,Close_ET_4pm,Close Value Apple,Close Value SPDR S&P 500 ETF Trust,Close Value Coinbase,RSI,Close_UTC_00_lag1,Close_UTC_00_lag3,Close_UTC_00_lag7,avg_sentiment_score_lag1,avg_sentiment_score_lag3,avg_sentiment_score_lag7,RSI_lag1,RSI_lag3,RSI_lag7
7,2022-01-18,0.253282,0.460446,0.457443,0.450648,0.407832,0.479741,0.766526,0.570916,0.472817,0.472452,0.445929,0.253282,0.253282,0.210110,0.704307,0.734104,0.000000
8,2022-01-19,0.208571,0.460968,0.447338,0.457716,0.375319,0.456985,0.756412,0.507714,0.457443,0.477212,0.466867,0.253282,0.253282,0.260047,0.570916,0.754715,1.000000
9,2022-01-20,0.199878,0.464479,0.454140,0.481679,0.359654,0.432981,0.764989,0.541855,0.447338,0.472817,0.481957,0.208571,0.253282,0.222628,0.507714,0.704307,1.000000
10,2022-01-21,0.170538,0.419462,0.468584,0.400700,0.340528,0.390879,0.645036,0.600662,0.454140,0.457443,0.466895,0.199878,0.253282,0.253282,0.541855,0.570916,0.705182
11,2022-01-22,0.172293,0.355626,0.389974,0.331274,0.340528,0.390879,0.645036,0.353625,0.468584,0.447338,0.472452,0.170538,0.208571,0.253282,0.600662,0.507714,0.734104


## Year, Month, Date Extraction
This may be useful for the models that do not capture temporal differences, such as Linear Regression.

In [ ]:
#df 1
df_1['Time_UTC'] = pd.to_datetime(df_1['Time_UTC'])
# Set 'Time_UTC' as the index
df_1.set_index('Time_UTC', inplace=True)
ds = df_1.index.to_series()
df_1['YEAR'] = ds.dt.year
df_1['MONTH'] = ds.dt.month
#df_1['DAY_OF_WEEK'] = ds.dt.dayofweek
df_1['DAY'] = ds.dt.day


#df 2
df_2['Time_UTC'] = pd.to_datetime(df_2['Time_UTC'])
# Set 'Time_UTC' as the index
df_2.set_index('Time_UTC', inplace=True)
ds = df_2.index.to_series()
df_2['YEAR'] = ds.dt.year
df_2['MONTH'] = ds.dt.month
#df_2['DAY_OF_WEEK'] = ds.dt
df_2['DAY'] = ds.dt.day

In [ ]:
df_1.head()

,Close_Avg,Close_UTC_00,Close_ET_4pm,Close Value Apple,Close Value SPDR S&P 500 ETF Trust,Close Value Coinbase,RSI,Close_UTC_00_lag1,Close_UTC_00_lag3,Close_UTC_00_lag7,RSI_lag1,RSI_lag3,RSI_lag7,YEAR,MONTH,DAY
Time_UTC,,,,,,,,,,,,,,,,
2022-01-08,0.457446,0.448601,0.450442,0.429417,0.525828,0.808318,0.172559,0.476972,0.517471,0.533485,0.220912,0.345547,0.000000,2022,1,8
2022-01-09,0.456446,0.429663,0.451505,0.429417,0.525828,0.808318,0.150561,0.448601,0.521923,0.551399,0.172559,0.397311,1.000000,2022,1,9
2022-01-10,0.457169,0.453001,0.438294,0.429599,0.523044,0.778704,0.265888,0.429663,0.476972,0.541158,0.150561,0.220912,0.636253,2022,1,10
2022-01-11,0.460943,0.445929,0.453174,0.455920,0.543399,0.828141,0.255382,0.453001,0.448601,0.531334,0.265888,0.172559,0.471683,2022,1,11
2022-01-12,0.478779,0.466867,0.486503,0.460018,0.549496,0.817906,0.333373,0.445929,0.429663,0.517471,0.255382,0.150561,0.345547,2022,1,12


In [ ]:
#save
df_1.reset_index(inplace=True)
df_2.reset_index(inplace=True)
df_1.to_csv('feature_engineered_data_1.csv', index = False)
df_2.to_csv('feature_engineered_data_2.csv', index = False)

In [ ]:
df_1.head()

,Time_UTC,Close_Avg,Close_UTC_00,Close_ET_4pm,Close Value Apple,Close Value SPDR S&P 500 ETF Trust,Close Value Coinbase,RSI,Close_UTC_00_lag1,Close_UTC_00_lag3,Close_UTC_00_lag7,RSI_lag1,RSI_lag3,RSI_lag7,YEAR,MONTH,DAY
0,2022-01-08,0.457446,0.448601,0.450442,0.429417,0.525828,0.808318,0.172559,0.476972,0.517471,0.533485,0.220912,0.345547,0.000000,2022,1,8
1,2022-01-09,0.456446,0.429663,0.451505,0.429417,0.525828,0.808318,0.150561,0.448601,0.521923,0.551399,0.172559,0.397311,1.000000,2022,1,9
2,2022-01-10,0.457169,0.453001,0.438294,0.429599,0.523044,0.778704,0.265888,0.429663,0.476972,0.541158,0.150561,0.220912,0.636253,2022,1,10
3,2022-01-11,0.460943,0.445929,0.453174,0.455920,0.543399,0.828141,0.255382,0.453001,0.448601,0.531334,0.265888,0.172559,0.471683,2022,1,11
4,2022-01-12,0.478779,0.466867,0.486503,0.460018,0.549496,0.817906,0.333373,0.445929,0.429663,0.517471,0.255382,0.150561,0.345547,2022,1,12
